## Introduzione 

Questa lezione coprirà: 
- Cos'è la chiamata di funzione e i suoi casi d'uso 
- Come creare una chiamata di funzione usando OpenAI 
- Come integrare una chiamata di funzione in un'applicazione 

## Obiettivi di apprendimento 

Dopo aver completato questa lezione saprai come e comprenderai: 

- Lo scopo dell'uso della chiamata di funzione 
- Configurare la chiamata di funzione utilizzando il servizio OpenAI 
- Progettare chiamate di funzione efficaci per il caso d'uso della tua applicazione


## Comprendere le Chiamate di Funzione

Per questa lezione, vogliamo costruire una funzionalità per la nostra startup educativa che permetta agli utenti di utilizzare un chatbot per trovare corsi tecnici. Consiglieremo corsi che si adattano al loro livello di competenza, ruolo attuale e tecnologia di interesse.

Per completare questo utilizzeremo una combinazione di:
 - `OpenAI` per creare un'esperienza di chat per l'utente
 - `Microsoft Learn Catalog API` per aiutare gli utenti a trovare corsi basati sulla richiesta dell'utente
 - `Function Calling` per prendere la query dell'utente e inviarla a una funzione per effettuare la richiesta API.

Per iniziare, vediamo perché vorremmo usare la chiamata di funzione in primo luogo:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # ottenere una nuova risposta da GPT dove può vedere la risposta della funzione


print(second_response.choices[0].message)


### Perché la Chiamata di Funzioni

Se hai completato qualche altra lezione in questo corso, probabilmente comprendi il potere dell'uso dei Modelli di Linguaggio di Grandi Dimensioni (LLM). Speriamo che tu possa anche vedere alcune delle loro limitazioni.

La Chiamata di Funzioni è una funzionalità del Servizio OpenAI progettata per affrontare le seguenti sfide:

Formattazione della Risposta Incoerente:
- Prima della chiamata di funzioni, le risposte di un modello di linguaggio di grandi dimensioni erano non strutturate e incoerenti. Gli sviluppatori dovevano scrivere codice di validazione complesso per gestire ogni variazione nell'output.

Integrazione Limitata con Dati Esterni:
- Prima di questa funzionalità, era difficile incorporare dati da altre parti di un'applicazione in un contesto di chat.

Standardizzando i formati di risposta e abilitando un'integrazione fluida con dati esterni, la chiamata di funzioni semplifica lo sviluppo e riduce la necessità di logiche di validazione aggiuntive.

Gli utenti non potevano ottenere risposte come "Qual è il meteo attuale a Stoccolma?". Questo perché i modelli erano limitati al tempo in cui i dati erano stati addestrati.

Guardiamo l'esempio qui sotto che illustra questo problema:

Supponiamo di voler creare un database di dati degli studenti così da poter suggerire loro il corso giusto. Qui sotto abbiamo due descrizioni di studenti che sono molto simili nei dati che contengono.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Vogliamo inviare questo a un LLM per analizzare i dati. Questo potrà poi essere utilizzato nella nostra applicazione per inviarlo a un'API o memorizzarlo in un database.

Creiamo due prompt identici in cui istruiamo l'LLM su quali informazioni ci interessano:


Vogliamo inviare questo a un LLM per analizzare le parti importanti per il nostro prodotto. Così possiamo creare due prompt identici per istruire l'LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Dopo aver creato questi due prompt, li invieremo al LLM utilizzando `openai.ChatCompletion`. Memorizziamo il prompt nella variabile `messages` e assegniamo il ruolo a `user`. Questo serve a simulare un messaggio scritto da un utente a un chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Ora possiamo inviare entrambe le richieste al LLM ed esaminare la risposta che riceviamo.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Anche se i prompt sono gli stessi e le descrizioni sono simili, possiamo ottenere formati diversi della proprietà `Grades`.

Se esegui la cella sopra più volte, il formato può essere `3.7` o `3.7 GPA`.

Questo perché il LLM prende dati non strutturati sotto forma di prompt scritto e restituisce anche dati non strutturati. Abbiamo bisogno di un formato strutturato in modo da sapere cosa aspettarci quando memorizziamo o utilizziamo questi dati.

Utilizzando il functional calling, possiamo assicurarci di ricevere dati strutturati in risposta. Quando si usa il functional calling, il LLM in realtà non chiama o esegue alcuna funzione. Invece, creiamo una struttura che il LLM deve seguire per le sue risposte. Usiamo quindi quelle risposte strutturate per sapere quale funzione eseguire nelle nostre applicazioni.


![Diagramma del flusso di chiamata della funzione](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.it.png)


Possiamo quindi prendere ciò che viene restituito dalla funzione e inviarlo indietro al LLM. Il LLM risponderà quindi utilizzando il linguaggio naturale per rispondere alla domanda dell'utente.


### Casi d'uso per l'utilizzo delle chiamate di funzione

**Chiamare Strumenti Esterni**  
I chatbot sono ottimi nel fornire risposte alle domande degli utenti. Utilizzando le chiamate di funzione, i chatbot possono usare i messaggi degli utenti per completare determinate attività. Ad esempio, uno studente può chiedere al chatbot di "Inviare un'email al mio insegnante dicendo che ho bisogno di più assistenza su questo argomento". Questo può fare una chiamata di funzione a `send_email(to: string, body: string)`


**Creare Query API o Database**  
Gli utenti possono trovare informazioni usando il linguaggio naturale che viene convertito in una query formattata o richiesta API. Un esempio potrebbe essere un insegnante che chiede "Chi sono gli studenti che hanno completato l'ultimo compito" che potrebbe chiamare una funzione chiamata `get_completed(student_name: string, assignment: int, current_status: string)`


**Creare Dati Strutturati**  
Gli utenti possono prendere un blocco di testo o CSV e usare l'LLM per estrarre informazioni importanti da esso. Ad esempio, uno studente può convertire un articolo di Wikipedia sugli accordi di pace per creare flashcard AI. Questo può essere fatto usando una funzione chiamata `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Creare la tua prima chiamata di funzione

Il processo di creazione di una chiamata di funzione include 3 passaggi principali:  
1. Chiamare l'API Chat Completions con un elenco delle tue funzioni e un messaggio dell'utente  
2. Leggere la risposta del modello per eseguire un'azione, cioè eseguire una funzione o una chiamata API  
3. Effettuare un'altra chiamata all'API Chat Completions con la risposta della tua funzione per utilizzare tali informazioni per creare una risposta per l'utente.


![Flusso di una Chiamata di Funzione](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.it.png)


### Elementi di una chiamata di funzione 

#### Input degli utenti 

Il primo passo è creare un messaggio utente. Questo può essere assegnato dinamicamente prendendo il valore di un input di testo oppure puoi assegnare un valore qui. Se è la prima volta che lavori con l'API Chat Completions, dobbiamo definire il `role` e il `content` del messaggio. 

Il `role` può essere `system` (creazione di regole), `assistant` (il modello) o `user` (l'utente finale). Per la chiamata di funzione, assegneremo questo come `user` e una domanda di esempio. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Creazione di funzioni.

Successivamente definiremo una funzione e i parametri di quella funzione. Useremo una sola funzione qui chiamata `search_courses` ma puoi creare più funzioni.

**Importante** : Le funzioni sono incluse nel messaggio di sistema per il LLM e saranno conteggiate nel numero di token disponibili.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definizioni** 

La struttura della definizione della funzione ha più livelli, ciascuno con le proprie proprietà. Ecco una suddivisione della struttura annidata:

**Proprietà della funzione di livello superiore:**

`name` - Il nome della funzione che vogliamo venga chiamata.

`description` - Questa è la descrizione di come funziona la funzione. Qui è importante essere specifici e chiari.

`parameters` - Un elenco di valori e formato che si desidera che il modello produca nella sua risposta.

**Proprietà dell'oggetto Parameters:**

`type` - Il tipo di dato dell'oggetto parameters (di solito "object").

`properties` - Elenco dei valori specifici che il modello utilizzerà per la sua risposta.

**Proprietà dei singoli parametri:**

`name` - Implicitamente definito dalla chiave della proprietà (es. "role", "product", "level").

`type` - Il tipo di dato di questo parametro specifico (es. "string", "number", "boolean").

`description` - Descrizione del parametro specifico.

**Proprietà opzionali:**

`required` - Un array che elenca quali parametri sono necessari per completare la chiamata della funzione.


### Effettuare la chiamata alla funzione  
Dopo aver definito una funzione, ora dobbiamo includerla nella chiamata all'API di completamento chat. Lo facciamo aggiungendo `functions` alla richiesta. In questo caso `functions=functions`.  

C'è anche un'opzione per impostare `function_call` su `auto`. Questo significa che lasceremo che sia il LLM a decidere quale funzione deve essere chiamata in base al messaggio dell'utente invece di assegnarla noi stessi.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Ora diamo un'occhiata alla risposta e vediamo come è formattata:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Puoi vedere che il nome della funzione viene chiamato e dal messaggio dell'utente, il LLM è stato in grado di trovare i dati per adattarsi agli argomenti della funzione.


## 3.Integrare le chiamate di funzione in un'applicazione. 


Dopo aver testato la risposta formattata dal LLM, ora possiamo integrarla in un'applicazione. 

### Gestire il flusso 

Per integrare questo nella nostra applicazione, seguiamo i seguenti passaggi: 

Per prima cosa, facciamo la chiamata ai servizi OpenAI e memorizziamo il messaggio in una variabile chiamata `response_message`. 


In [ ]:
response_message = response.choices[0].message

Ora definiremo la funzione che chiamerà l'API di Microsoft Learn per ottenere un elenco di corsi:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Come buona pratica, vedremo quindi se il modello vuole chiamare una funzione. Successivamente, creeremo una delle funzioni disponibili e la abbineremo alla funzione che viene chiamata.  
Prenderemo quindi gli argomenti della funzione e li mapperemo agli argomenti provenienti dal LLM.

Infine, aggiungeremo il messaggio di chiamata della funzione e i valori restituiti dal messaggio `search_courses`. Questo fornisce al LLM tutte le informazioni di cui ha bisogno per rispondere all'utente usando il linguaggio naturale.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Ora invieremo il messaggio aggiornato al LLM in modo da poter ricevere una risposta in linguaggio naturale invece di una risposta formattata in JSON API.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Sfida di Codice 

Ottimo lavoro! Per continuare il tuo apprendimento su OpenAI Function Calling puoi costruire: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - Più parametri della funzione che potrebbero aiutare gli studenti a trovare più corsi. Puoi trovare i parametri API disponibili qui: 
 - Crea un'altra chiamata di funzione che prenda più informazioni dallo studente come la loro lingua madre 
 - Crea una gestione degli errori quando la chiamata di funzione e/o la chiamata API non restituiscono corsi adatti 


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Pur impegnandoci per garantire l’accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa deve essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un umano. Non ci assumiamo alcuna responsabilità per eventuali malintesi o interpretazioni errate derivanti dall’uso di questa traduzione.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
